<a href="https://colab.research.google.com/github/bogues26/bogues26/blob/main/Copia_de_Puntua_headings_enlaces_SERP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrapea y puntúa headings y enlaces:
=> Este colab scrapea los artículos de la SERP y puntua sus headings/enlaces por relevancia con la keyword

1. Cambia en el código la keyword por lo que quieras (ej. keyword = "tipos de fiebre")
2. Pulsa el play
3. Espera a que el script acabe

Debajo del código aparecerán 2 tablas con las puntuaciones



---



Un saludo desde Malloca,

Jose Gris


In [ ]:
#Cambia la keyword por la que quieras (ej. keyword = "mejores cafeteras 2022")
keyword = "dieta keto"
encabezados = ['h1', 'h2', 'h3']

import requests
from bs4 import BeautifulSoup
import re
import nltk

class pagina():
    def __init__(self, url):
        self.url = url
        self.texto = ""
        self.enlaces  = []
        self.headings = []

class serp():
     
    # init method or constructor
    def __init__(self, query):
        self.query = query.replace(" ", "+")
        self.ok = False
        self.incidendias = []
        self.paginas = []
        self.start()

    def start(self):
        URL = "https://www.google.com/search?hl=es&gl=es&q=%s&oq=%s" % (self.query, self.query)
        headers =  {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36"}
        enlacesSerp = []
        resp = requests.get(URL, headers = headers)
        if resp.status_code == 200: 
            soup = BeautifulSoup(resp.content, "html.parser")
            links = soup.find_all("div", {"class" : "g"})
            print("Procesando urls:")
            for x in links:
                link = x.find("a")['href']
                #corrijo feature snippet
                if "#:~:text" in link: 
                    link = link.split("#:~:text")[0]
                if link.startswith("/"):continue
                print(link)

                if not link in enlacesSerp:
                    enlacesSerp.append(link)
                    
                    try:
                      resp = requests.get(link, headers = headers, timeout=10)
                         
                    except Exception as e:
                      self.incidendias.append(f"timeout error: {str(e)} url: {link}")

                    soup = BeautifulSoup(resp.content, "html.parser")

                    if resp.status_code == 200:
                        pag = pagina(link)
                        
                        #elimino scrips
                        for s in soup.find_all(['script', 'style']):
                            s.extract()
                        
                        #recupero texto
                        texto = ""
                        tag = soup.body
                        for string in tag.strings:
                            texto = texto + " " + string
                        pag.texto = texto

                        #recupero enlaces
                        for en in soup.find_all("a", href=True):
                            if "#" in en["href"] or "http" in en["href"]:continue
                            if en.text not in pag.enlaces:
                                pag.enlaces.append(en.text)

                        #recupero headings
                        for heading in soup.find_all(encabezados):
                            if heading.text not in pag.headings:
                                pag.headings.append([heading.name, heading.text.strip('\n').strip()])
                        
                        self.paginas.append(pag)
                    else:
                        self.incidendias.append(f"Status code: {resp.status_code} url: {link}")

            if len(self.paginas) < 5:
                self.ok = True
                self.incidendias.append("Menos de 5 resultados escrapeados")
            else:
                self.ok = True
 
        else:
            self.ok = False

        #reporto incidencias scrapeo
        print(str(len(self.paginas)) + " páginas escrapeadas correctamente")

        if len(self.incidendias) > 0:
            print("Incidencias")
            print("-----------")
            for x in self.incidendias:
                print(x)

#Aviso para ti, navegante programador => me da pereza limpiar el código 😉
scrap = serp(keyword)

if scrap.ok == False:
    print("No se ha completado el scrap, se detiene el análisis => Sorry...")
    quit()

#Creo el modelo word2Vec => paso de palabras a vectores
print("Instalando dependencias...")
!pip install nltk
!pip install gensim
from gensim.models import Word2Vec
from unicodedata import normalize
import spacy
nltk.download('stopwords')
nltk.download('punkt')
!python -m spacy download es_core_news_sm
import es_core_news_sm
nlp = es_core_news_sm.load()

print("Pasando texto scrapeado a vectores y creando modelo word2Vec...")
texto = ""
for p in scrap.paginas:
    texto = texto + " " + p.texto
texto = texto.strip()

def lematizarFrase(frase):
    doc = nlp(frase)
    lematizado = [token.lemma_ for token in doc]
    return " ".join(lematizado)

def quitarAcentos(frase):
  frase = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", frase), 0, re.I
    )
  frase = normalize( 'NFC', frase)
  return frase

from nltk.corpus import stopwords
def quitarStopWords(frase):
  tokenizado = frase.split()
  tokenizado = [w for w in tokenizado if w not in stopwords.words('spanish')]
  return " ".join(tokenizado)

def limpiarBasico(frase, lemat = True):
    frase = re.sub('[^a-zA-ZÀ-ÿ\u00f1\u00d1]', ' ', frase)
    frase = re.sub(r'\s+', ' ', frase)
    frase = frase.replace("  ", " ").strip()
    if lemat == True: frase = lematizarFrase(frase)
    frase = quitarAcentos(frase)
    return frase

def limpia(texto):
    procesando = limpiarBasico(texto)
    procesando = procesando.lower()
    #procesando = quitarAcentos(procesando)
    procesando = procesando.replace("(", "").replace(")", "")
        
    frases = nltk.sent_tokenize(procesando)
    tokenizado = [nltk.word_tokenize(sent) for sent in frases]

    # Removing Stop Words
    for i in range(len(tokenizado)):
        tokenizado[i] = [w for w in tokenizado[i] if w not in stopwords.words('spanish')]
    return tokenizado

textoLimpio = limpia(texto)
from gensim.models import Word2Vec

word2vec = Word2Vec(textoLimpio, min_count=2, iter= 2)
modelo = word2vec.wv
print("Modelo creado => Puntuando headings/enlaces...")

#modelo.vocab
vocabulario = modelo.index2word

def limpiarFrase(frase):
    frase = frase.lower()
    frase = frase.replace("(", "").replace(")", "")
    frase = limpiarBasico(frase)
    frase = frase.split()
    frase = [x for x in frase if x in vocabulario]   
    return " ".join(frase).strip()

headings = []
for pagina in scrap.paginas:
    for heading in pagina.headings:
      he = []
      
      he.append(heading[1].strip())
      sinStop = quitarStopWords(heading[1])
      he.append(limpiarFrase(sinStop))
      headings.append(he)

enlaces = []
for pagina in scrap.paginas:
    for enlace in pagina.enlaces:
      he = []
      he.append(enlace.strip())
      sinStop = quitarStopWords(enlace)
      he.append(limpiarFrase(sinStop))
      enlaces.append(he)


headings = [hea for hea in headings if hea[1].strip() !=""]
enlaces = [hea for hea in enlaces if hea[1].strip() !=""]

keys = [x for x in limpiarBasico(keyword).lower().split() if x in vocabulario]

#puntuo los enlaces/headings por relevancia con la keyword
def puntuar(keyword, lista):
  for en in lista:
    palabrasRelev = [palabra for palabra in en[1].split()]
    res = round(modelo.n_similarity(keys, palabrasRelev),3)
    en.append(str(res))
  lista = sorted(lista, key=lambda x: x[2], reverse=True)
  return lista
  

headings = puntuar(keyword, headings)
enlaces = puntuar(keyword,enlaces)


def agrupar(lista0):
  lista = lista0.copy()
  headingsAgrupados = []
  while len(lista)>0:
    relacionados = []
    primero = lista.pop(0)
    palabrasPrimero = [palabra for palabra in primero[1].split()]
    
    for hea in list(lista):
      palabrasRelev = [palabra for palabra in hea[1].split()]
      res = round(modelo.n_similarity(palabrasPrimero, palabrasRelev),3)
      
      if res == 1:
        #print(str(res), palabrasPrimero, palabrasRelev)
        if hea[0] not in relacionados and hea[0] != primero[0]:
          relacionados.append(hea[0])
        lista.remove(hea)
    relacionados = " - ".join(relacionados)
    primero.append("")
    primero.append(relacionados)
    headingsAgrupados.append(primero)
  return headingsAgrupados

#Agrupo los headings similares
headingsFinal = agrupar(headings)

#borro duplicados enlaces
enlaces2 = []
for enlace in enlaces:
  anchors = [x[0].lower() for x in enlaces2]
  if enlace[0].lower() not in anchors:
    enlaces2.append(enlace)
enlaces = enlaces2

#Paso de listas a tablas
import pandas as pd 

headingsFinal = [heading[:5:2] for heading in headingsFinal]
enlacesFinal = [enlace[::2] for enlace in enlaces]

#Visualizo tablas
from google.colab import data_table
def pasarATabla(lista,columnas):
  lista = pd.DataFrame (lista, columns = columnas )
  lista = data_table.DataTable(lista, include_index=True, num_rows_per_page=20)
  display(lista)


pasarATabla(headingsFinal, ['Encabezado', 'Puntuación', 'Agrupados'])
pasarATabla(enlacesFinal, ['Enlaces', 'Puntuación'])

print("-----------------------------------------")
print("Que la fuerza te acompañe joven Skywalker")
print("-----------------------------------------")

#Si quieres visualizar el vocabulario => quita el # de la siguiente línea y dale al play.
#pasarATabla(vocabulario, ['Vocabulario'])

Procesando urls:
https://www.tucanaldesalud.es/es/voz-especialista/dieta-keto-solucion-perdida-peso
https://www.directoalpaladar.com/salud/que-dieta-keto-quien-puede-seguirla-41-recetas-que-se-adaptan-a-ella
https://nutricionyfarmacia.es/blog/dietetica/dietas/dieta-keto/
https://www.dietdoctor.com/es/keto
https://www.topdoctors.es/diccionario-medico/dieta-cetogenica
https://www.lavanguardia.com/comer/al-dia/20210813/7660110/estudio-revela-quienes-deberian-seguir-nunca-dieta-keto.html
https://www.sabervivirtv.com/nutricion/que-es-dieta-keto-cetogenica_5285
https://www.vitonica.com/dietas/menu-14-dias-comidas-cenas-completas-para-empezar-dieta-cetogenica
8 páginas escrapeadas correctamente
Instalando dependencias...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 16.2 MB 3.2 MB/s 
 

,Encabezado,Puntuación,Agrupados
0,Dieta keto para principantes,1.0,1. ¿Qué es una dieta keto? - Una dieta keto má...
1,Guías visuales de la dieta keto,0.919,6. Guías prácticas de dieta keto
2,Desventajas de la dieta keto,0.915,
3,Información sobre la dieta keto en 40 idiomas,0.91,
4,Riesgos de la dieta keto,0.908,
...,...,...,...
115,Aprende a preparar el postre de temporada más ...,-0.064,
116,Programas y cursos,-0.055,
117,Qué beber,-0.04,
118,Epilepsia,-0.032,


,Enlaces,Puntuación
0,Dieta keto,1.0
1,Dieta Keto (cetogénica),0.902
2,"Dieta Keto, ¿una solución para la pérdida de p...",0.828
3,Qué es la dieta cetogénica (keto) y cuándo es ...,0.81
4,dieta,0.801
...,...,...
469,Cuidados para dientes con sensibilidad dental,-0.007
470,test Covid-19,-0.003
471,Covid persistente,-0.003
472,Consulta Inmediata,-0.002


-----------------------------------------
Que la fuerza te acompañe joven Skywalker
-----------------------------------------


↑ Las headings y enlaces puntuados aparecerán arriba => Puedes pulsar el botón de copiar en cada tabla para descargarlas

↓ Si quieres descargarte los headings de cada artículo => Pulsa al play en el código de abajo

In [ ]:
# Headings de cada artículo

ruta = f'{keyword} - headings.txt'
file = open(ruta, 'w', encoding="utf-8")

file.write('Keyword: ' + keyword+"\n\n")
for p in scrap.paginas:
    file.write(p.url+"\n")
    for h in p.headings:
        Ntab = "  " * int(h[0].replace("h", "").replace("H", ""))
        file.write(f"{Ntab}{h[0]} - {h[1]}\n")
    file.write("\n")

from google.colab import files
files.download(ruta)


↑ La puntuación de headings/enlaces se muestra más arriba ↑